In [1]:
import pandas as pd
import numpy as np
import datetime
import sys, os
import glob
import time
import fnmatch
import os
import xlwings as xw
start_time = time.time()

print('Reading the Current Backlog Combined Data')
df1 = pd.read_excel("dataEVM.xlsx", header=0, dtype={'PART': str})
#df1 = pd.read_excel("dataEVM.xlsx", header=2, dtype={'PART': str,'SSD': str})

df1['SSD'] = pd.to_datetime(df1['SSD'])
df1['MATL_TYPE']=np.where(df1['MATL_TYPE']== 'Included','Simple',df1['MATL_TYPE'])
#df1.MATL_TYPE.unique()

df1['Material Avail Date > CRSD'] = np.where(df1['SSD'] > df1['CRSD'], 'Yes', 'No') 

df1['SHIP_LOC'] = df1['SHIP_LOC'].str[:3] #delete possible dc with an -

df1['SHIP_TO'] = df1['SOLD_TO'] #Copy AE column to AF

print('Filtering file for EVM')
df1 = df1[(df1['EVM_AIT2'] == 'EVM')]
#df1 = df1[(df1['SHIP_LOC'] != 'FL1') & (df1['SHIP_LOC'] != 'UK1') & (df1['SHIP_LOC'] != 'VH1')]
df1 = df1[(df1['SHIP_LOC'] != 'UK1') & (df1['SHIP_LOC'] != 'VH1')]
print('Inserting columns')
df1 = df1.drop(['QOH'], axis = 1)
df1.rename(columns={'NETTABLE_QOH': 'QOH'},inplace=True)
df1['SSD'] = pd.to_datetime(df1['SSD'])
df1.insert(2, 'Trilogy Or Merlin', 'MERLIN')
#df1.insert(3, 'Material Avail Date > CRSD', '')
#df1.insert(4, 'SO Line Dropped to DC for Delivery', '')
df1.insert(5, 'SKU Short(QTYs On Hand) DBL & CRSD <=+4', '')
df1.insert(6, 'SO Short(QTYs On Hand) DBL & CRSD <=+4', '')
#df1.insert(7, 'Missing CRSD Wk', '')
df1.insert(8, 'CRSD Days', '')
df1.insert(9, 'TPC SKU/DC/WK Key', '')
df1.insert(10, 'Missin Commits TPC/ZCM WK2', '')
df1["SKU DC Key"]=df1["PART"]+df1["SHIP_LOC"]
df1.insert(13, 'Blended ASP $', '')
df1.insert(14, 'EXT Part ASP $', '')
df1.insert(19, 'Oracle Intransit(Trilogy IR/ISO)', '')
df1.insert(20, 'Oracle Intransit(Trilogy=>Merlin or Merlin=>Trilogy)', '')
df1.insert(21, 'ZCM ASN QTY Supply', '')
df1.insert(22, 'ZCM Commits WK1', '')
df1.insert(23, 'ZCM Commits WK2', '')
df1.insert(24, 'ZCM Commits WK3', '')
df1.insert(25, 'ZCM Commits WK4', '')
df1.insert(26, 'ZCM Commits WK5', '')
df1.insert(27, 'WK1 TPC Receipt QTY', '')
df1.insert(28, 'Valid TPC Supply', '')
df1.insert(29, 'TPC Commits WK1', '')
df1.insert(30, 'TPC Commits WK2', '')
df1.insert(31, 'TPC Commits WK3', '')
df1.insert(32, 'TPC Commits WK4', '')
df1.insert(34, 'ZCM ASN QTY and WK1 Net Supply', '')
df1.insert(35, 'ZCM Net Commits WK2', '')
df1.insert(36, 'ZCM Net Commits WK3', '')
df1.insert(37, 'Valid TPC Net Supply WK1', 0) #CS
df1.insert(38, 'TPC Net Commits WK2', 0) #CT
df1.insert(39, 'TPC Net Commits WK3', 0) #CU
df1.rename(columns={'SOURCE': 'SOURCE.'},inplace =True) #COLUMNA AZUL EN EXCEL, IE NO SE MANIPULARA
df1.insert(41, 'Net Supply capture by CRSD SKU/DC/WK', '')
df1.insert(42, 'Missing ASN Supply Due to CRSD WK', '')
df1.insert(43, 'Missing Commit Supply Due to CRSD WK', '')
df1.insert(44, 'Merlin/Trilogy Missing Supply Total', '')
df1.insert(45, 'SKU/DC Total Supply(OH only)', '')
df1.insert(46, 'QTY Short to Sales Orders(CUM) Only INV', '')
df1.insert(47, 'QTY Short to Sales Orders(NON-CUM) Only INV', '')
df1.insert(48, 'EXT Part ASP $ (QTY Short to Sales Orders(NON-CUM) Only INV)', '')
df1.insert(49, 'SKU/DC Total Supply(On Hand+ALL ASN)', '')
df1.insert(50, 'QTY Short to Sales Orders(CUM) (On Hand+ALL ASN', '')
df1.insert(51, 'QTY Short to Sales Orders(NON-CUM) On Hand+ALL ASN', '')
df1.insert(52, 'EXT Part ASP $ (QTY Short to Sales Orders(NON-CUM)) On Hand+ALL ASN', '')
df1.insert(53, 'SKU/DC Total Supply ALL ASNs+Commits(No Missing Commits)', '')
df1.insert(54, 'QTY Short to Sales Orders(CUM) including Missing Commits+Commits+ALL ASNs', '')
df1.insert(55, 'QTY Short to Sales Orders(NON-CUM) w/ Commits+Missing Commits+ALL ASNs', '')
df1.insert(56, 'EXT Part ASP $ (QTY Short to Sales Orders(NON-CUM)) INV w/ Commits+Missing Commits+ALL ASNs', '')
df1.insert(57, 'SKU/DC Total Supply(OH+ALL ASNs+all 3wk Commtsh)', '')
df1.insert(58, 'QTY Short to Sales Orders(CUM) OH+ALL ASNs+all 3wk Commts', '')
df1.insert(59, 'QTY Short to Sales Orders(NON-CUM) OH+ALL ASNs+all 3wk Commts', '')
df1.insert(60, 'EXT Part ASP $ (QTY Short to Sales Orders(NON-CUM)) OH+ALL ASNs+all 3wk Commts', '')
len(df1)

#lookup Delivery Creation Date
print('Reading the In Play Report')
df2 = pd.read_csv('In Play Report.csv')
df2 = df2[['SO_LINE','Delivery Creation Date']]
df2=df2.drop_duplicates(subset='SO_LINE',keep='first') #df.drop_duplicates as some SO LINES have 2 dates
#first was kept since thats how it was done in excel
df1 = pd.merge(df1, df2, on='SO_LINE', how='left')
df1.rename(columns={'Delivery Creation Date':'SO Line Dropped to DC for Delivery'}, inplace=True)
#df1['SO Line Dropped to DC for Delivery']=df1['SO Line Dropped to DC for Delivery'].str[:-5]
df1['SO Line Dropped to DC for Delivery']=pd.to_datetime(df1['SO Line Dropped to DC for Delivery'])

#lookup if it is a ZCM Item 
print('Reading ZCM Item Report')
#df3 = pd.read_excel("ZCM Items.xlsx", dtype={'SKU DC': str})
#df3 = df3[['SKU DC','ZCM']]
#df3.rename(columns={'SKU DC': 'SKU DC Key','ZCM':'TPC/ZCM Transmit'},inplace=True)

df3 = pd.read_excel("ZCM Items.xlsx", dtype={'Zebra PN': str,'Zebra Site':str})
df3['SKU DC Key'] = df3['Zebra PN'] + df3['Zebra Site']
df3['TPC/ZCM Transmit'] = "Yes"
df3=df3[['SKU DC Key','TPC/ZCM Transmit']]
df3=df3.drop_duplicates()

df1 = pd.merge(df1, df3, on='SKU DC Key', how='left')
df1['TPC/ZCM Transmit'].fillna('No', inplace = True)

#Service Offering
#lookup ASP
print('Reading Fill Rate')
df4 = pd.read_excel("Fill Rate.xlsx",dtype={'Part No': str, 'ASP_Original': float})
df4 = df4[['Part No','ASP_Original']]
df4.rename(columns={'Part No': 'PART','ASP_Original':'Part ASP $'},inplace=True)
df4=df4.drop_duplicates(subset='PART',keep='first')
df1 = pd.merge(df1, df4, on='PART', how='left')
df1['Part ASP $'].fillna(0, inplace=True)

#lookup Sourcing Rule, Buyer, Planner Code
print('Reading the Sourcing Rules for EVM')
df5 = pd.read_csv('Sourcing Rule EVM.csv', dtype={'SKU DC': str})
df5 = df5[['SKU DC','Item Organization - Sourcing Rule','Default Buyer','Planner Code']]
df5.rename(columns={'SKU DC': 'SKU DC Key','Item Organization - Sourcing Rule':'Sourcing Rule',
                          'Default Buyer':'BUYER.','Planner Code':'PLANNER CODE'},inplace=True)
df1 = pd.merge(df1, df5, on='SKU DC Key', how='left')
#df1['Sourcing Rule'].fillna(0, inplace=True)#left as empty to replace nulls with other data
df1['BUYER.'].fillna(0, inplace=True)
df1['PLANNER CODE'].fillna(0, inplace=True) 

#df1.loc[df1['SKU DC Key'] == "RFD8500-1000100-USDS2"]

#FILL NAs from Sourcing Rule WITH AGILE LIST
print('Reading Agile List')
dfagile = pd.read_excel("Agile List.xlsx", dtype={'Item Number': str})
df6 = dfagile[['Item Number','Manufacturer Name']]
df6.rename(columns={'Item Number': 'PART'},inplace =True)
df6=df6.drop_duplicates(subset='PART',keep='first') #df.drop_duplicates
#first was kept since thats how it was done in excel
df1 = pd.merge(df1, df6, on='PART', how='left')
df1['Sourcing Rule'] = df1['Sourcing Rule'].fillna(df1['Manufacturer Name'])#where sourcing rule is empty
#this values change to the result of the merge, ie the manufacturer name in agile file
df1 = df1.drop(['Manufacturer Name'], axis = 1) #after gettin these values, this column is no longer needed
len(df1)

#create an auxiliar table with no sourcing rule in agile to look up in Agile EVM
dfSourcingRule=df1[df1['Sourcing Rule'].isnull()]
dfSourcingRule=dfSourcingRule[['PART','Sourcing Rule']]
dfSourcingRule=dfSourcingRule.drop_duplicates() #CREATE NEW INDEXES
dfSourcingRule.reset_index(inplace=True)
dfSourcingRule.drop("index",axis=1,inplace=True)

len(dfSourcingRule) #HOW MANY PARTS WE'LL LOOK FOR IN AGILE EVM
for i in dfSourcingRule.index:
 a=dfSourcingRule['PART'][i]
 print ("Look the next part number in Agile EVM",{a})
 dfSourcingRule['Sourcing Rule'][i] = input()

dfSourcingRule.rename(columns={'Sourcing Rule':'Sourcing Rule1'},inplace=True)
df1 = pd.merge(df1, dfSourcingRule, on='PART', how='left')
df1['Sourcing Rule'] = df1['Sourcing Rule'].fillna(df1['Sourcing Rule1'])
df1 = df1.drop(['Sourcing Rule1'], axis = 1)
#FILL NAN WITH INFO FOUND IN AGILE EVM

#create XFR column
df1['XFR'] = np.where((df1['Sourcing Rule'].isnull()), '',
             np.where(df1['Sourcing Rule'].str.contains('Transfer'), 'Yes',
             np.where(df1['Sourcing Rule'].str.contains('transfer'), 'Yes', '')))

#if Sourcing Rule starts with Make turn MATL_TYPE to bundle
#df1['MATL_TYPE'] = np.where(df1['Sourcing Rule'].astype(str).str.startswith('Make'), 'Bundle', df1['MATL_TYPE'])

#if MATL_TYPE is Bundle make PART ASP $, UNIT_VAL, TOT_BL all zero
df1['Part ASP $'] = np.where(df1['MATL_TYPE'].astype(str).str.startswith('Bundle'), 0, df1['Part ASP $'])
df1['UNIT_VAL'] = np.where(df1['MATL_TYPE'].astype(str).str.startswith('Bundle'), 0, df1['UNIT_VAL'])
df1['TOT_BL'] = np.where(df1['MATL_TYPE'].astype(str).str.startswith('Bundle'), 0, df1['TOT_BL'])

#create Blended ASP and EXT Part ASP
df1['Blended ASP $'] = np.where((df1['UNIT_VAL'] > 0), df1['UNIT_VAL'], df1['Part ASP $'])
df1['EXT Part ASP $'] = df1['ACTUAL_QTY'] * df1['Blended ASP $']

#lookup Oracle Intransit(Trilogy=>Merlin or Merlin=>Trilogy)
print('Reading Inter-Org Intransit')
df7 = pd.read_csv("Inter-Org Intransit.csv", dtype={'ITEM': str})
df7['SKU DC Key'] = df7['ITEM'] + df7['ORG'].str[4:]
df7 = pd.pivot_table(df7, values=['DEMAND_SUPPLY_QTY'], index=['SKU DC Key'], aggfunc=np.sum)
df1 = pd.merge(df1, df7, on='SKU DC Key', how='left')
df1['Oracle Intransit(Trilogy=>Merlin or Merlin=>Trilogy)'] = df1['DEMAND_SUPPLY_QTY']
df1['Oracle Intransit(Trilogy=>Merlin or Merlin=>Trilogy)'].fillna(0, inplace=True)

df1 = df1.drop(['DEMAND_SUPPLY_QTY'], axis = 1)
len(df1)

#Merge shipment files together
import glob
print("Reading the shipments")
path = r'C:\Users\MJ7255\Desktop\Backlog\INSUMOS ASN' # use your path
all_files = glob.glob(path + "/*.xlsx")

li = []

for filename in all_files:
    df = pd.read_excel(filename, index_col=None, header=1)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
#export ASN file
from datetime import date
#today = date.today()#tday = str(today.strftime("%m/%d/%y")) #tday='ASNs ' + tday #today
#frame.to_excel('ASNs '+str(date.today())+'.xlsx', index=False)
frame=frame[['*Zebra PN','*Ship To Site','*Shipped Quantity']]
frame['SKU DC Key']=frame["*Zebra PN"] + frame["*Ship To Site"]
df11 = pd.pivot_table(frame, values=['*Shipped Quantity'], index=['SKU DC Key'], aggfunc=np.sum)
df1 = pd.merge(df1, df11, on='SKU DC Key', how='left')
df1['ZCM ASN QTY Supply'] = df1['*Shipped Quantity']
df1['ZCM ASN QTY Supply'].fillna(0, inplace=True)
df1 = df1.drop(['*Shipped Quantity'], axis = 1)
print("ASN file created")
#df11.to_excel('ASN PIVOT.xlsx')

#lookup weekly ZCM Commits
print('Reading All Commit_ZCM')
a=fnmatch.filter(os.listdir('.'), 'All Commit_ZCM*')
a=a[0]

#df12.head()
#df1 = df1.drop(['ALL MISSING','Week 1','Week 2','ALL WK 1 & 2'], axis = 1)

df12 = pd.read_excel(a, header=1, sheet_name='ZCM COMMITS', dtype={'SKU DC': str})
df12 = df12[(df12['Supplier ID'] != 'Supplier ID')] #subset rows with info, there some rows with wronf information, headers in ros and dates in week info
df12 = df12.rename(columns={'SKU DC': 'SKU DC Key'})
df12 = df12[['SKU DC Key','ALL MISSING','ALL WK 1 & 2','Data Measure','Week 1','Week 2','Week 3','Week 4','Week 5']]

df12 = df12.groupby(['SKU DC Key']).sum()
df12.sort_values(by='ALL MISSING', ascending=False, inplace=True)

df1 = pd.merge(df1, df12, on='SKU DC Key', how='left')
df1['ZCM Commits WK1'] = df1['Week 1']
df1['ZCM Commits WK1'].fillna(0, inplace=True)
df1['ZCM Commits WK2'] = df1['Week 2']
df1['ZCM Commits WK2'].fillna(0, inplace=True)
df1['ZCM Commits WK3'] = df1['Week 3']
df1['ZCM Commits WK3'].fillna(0, inplace=True)
df1['ZCM Commits WK4'] = df1['Week 4']
df1['ZCM Commits WK4'].fillna(0, inplace=True)
df1['ZCM Commits WK5'] = df1['Week 5']
df1['ZCM Commits WK5'].fillna(0, inplace=True)

df1 = df1.drop(['ALL MISSING','Week 1','Week 2','Week 3','Week 4','Week 5'], axis = 1)

#lookup to see if items are Xplore
print('Reading Xplore Items')
df14 = pd.read_excel("Xplore Items.xlsx", sheet_name='Sheet1', dtype={'Number': str})
df14 = df14.rename(columns={'Number': 'PART'})
df14 = df14[['PART','MAKE']]
df1 = pd.merge(df1, df14, on='PART', how='left')
df1['Xplore Make items'] = df1['MAKE']
df1['Xplore Make items'].fillna('No', inplace=True)
df1 = df1.drop(['MAKE'], axis = 1)

#create week functions
today = date.today()
coming_sunday = today + datetime.timedelta(days=-today.weekday()-1, weeks=1) #this sunday sunday 1
coming_sunday = coming_sunday.strftime('%Y-%m-%d') #format it
coming_monday = today + datetime.timedelta(days=-today.weekday(), weeks=1)#this monday monday 1
coming_monday = coming_monday.strftime('%Y-%m-%d') #format it
next_sunday = today + datetime.timedelta(days=-today.weekday()-1, weeks=2)#sunday 2 
next_sunday = next_sunday.strftime('%Y-%m-%d') #format it
next_monday = today + datetime.timedelta(days=-today.weekday(), weeks=2)#monday2
next_monday = next_monday.strftime('%Y-%m-%d') #format it
following_sunday = today + datetime.timedelta(days=-today.weekday()-1, weeks=3) #sunday 3
following_sunday = following_sunday.strftime('%Y-%m-%d') #format it

mask1 = (df1['CRSD'] <= coming_sunday) #filter for wk1
mask2 = (df1['CRSD'] >= coming_monday) & (df1['CRSD'] <= next_sunday) #filter for wk2
mask3 = (df1['CRSD'] >= next_monday) & (df1['CRSD'] <= following_sunday) #filter for wk3

#define in which week the CRSD is, (Week 1, 2, or 3)
df1['CRSD WK'] = np.where(mask1, 'WK1',
                 np.where(mask2, 'WK2',
                 np.where(mask3, 'WK3','')))

#JUST LEAVE VALUE IN WKS WHEN MAT TPYE IS SIMPLE
df1['CRSD WK']=np.where(df1['MATL_TYPE']== 'Simple',df1['CRSD WK'],'')

#COLUMN AB, BLUE, COLUMN USUALLY DONE/COPIED AT THE SAME TIME AS RED COLUMN
df1['TPC SKU/DC/WK Key'] = df1["SKU DC Key"] + df1["CRSD WK"]

#create the Material Avail Date > CRSD column, COLUMN K
df1['CRSD'] = df1['CRSD'].dt.strftime('%#m/%d/%Y')

df1['SSD']= df1['SSD'].dt.strftime('%m-%d-%Y')

#create the CRSD Days column, COLUMN J
df1['DATE'] = pd.to_datetime(df1['DATE'])
df1['CRSD'] = pd.to_datetime(df1['CRSD'])
df1['CRSD Days'] = (df1['DATE'] - df1['CRSD']).dt.days

#create Short(QTYs On Hand) DBL & CRSD <=+4 columns
df15 = df1[['SHIP_LOC','Trilogy Or Merlin','CRSD','SO','LINE','SO_LINE','SKU DC Key','MATL_TYPE','ACTUAL_QTY','QOH']]
#Select just simple
df15 = df15[df15['MATL_TYPE'] == 'Simple']
#auixiliar of dates
today = date.today()
four_days = today + datetime.timedelta(days=3)
four_days = four_days.strftime('%Y-%m-%d')
#when is less than five days
df15 = df15[df15['CRSD'] <= four_days]
#auxiliar so it can make a count with the 2nd column
df15['QOH_2'] = df15['QOH']

#create SKU Short(QTYs On Hand) DBL & CRSD <=+4 PIVOT
df16 = pd.pivot_table(data=df15, index=['SKU DC Key'], values=['QOH','QOH_2','ACTUAL_QTY'], aggfunc={'QOH': 'sum', 'QOH_2': 'count', 'ACTUAL_QTY': 'sum'}).reset_index()
df16['SKU/DC QOH'] = df16['QOH'] / df16['QOH_2']
df16['SKU/DC QOH'] = df16['QOH'] / df16['QOH_2']
df16['Short +/-'] = df16['SKU/DC QOH'] - df16['ACTUAL_QTY']
df16['SKU Short'] = np.where(df16['Short +/-'] <= 0, 'YES', 'NO')
df16 = df16[['SKU DC Key','SKU Short']] #Just keep columns we want
df1 = pd.merge(df1, df16, on='SKU DC Key', how='left')
df1['SKU Short(QTYs On Hand) DBL & CRSD <=+4'] = df1['SKU Short']
df1 = df1.drop(['SKU Short'], axis = 1)
df1['SKU Short(QTYs On Hand) DBL & CRSD <=+4'] = np.where((df1['CRSD'] <= four_days) & (df1['MATL_TYPE'] == 'Simple'), df1['SKU Short(QTYs On Hand) DBL & CRSD <=+4'], '')

#create SO Short(QTYs On Hand) DBL & CRSD <=+4 PIVOT
df15 = pd.merge(df15, df16, on='SKU DC Key', how='left')
df17 = pd.crosstab(index=df15["SO"], columns=df15["SKU Short"], values=df15["SKU DC Key"], aggfunc='count').reset_index()
df17['SO Short'] = np.where(df17['YES'].isnull(), 'NO', 'YES')
df17 = df17[['SO','SO Short']]

df1 = pd.merge(df1, df17, on='SO', how='left')
df1['SO Short(QTYs On Hand) DBL & CRSD <=+4'] = df1['SO Short']
df1 = df1.drop(['SO Short'], axis = 1)
df1['SO Short(QTYs On Hand) DBL & CRSD <=+4'] = np.where((df1['CRSD'] <= four_days) & (df1['MATL_TYPE'] == 'Simple'), df1['SO Short(QTYs On Hand) DBL & CRSD <=+4'], '')

#create ZCM ASN/Commit Net WK1 supply
df1['day_of_wk'] = df1['DATE'].dt.day_name()
df1['ZCM ASN/Commit Net WK1 Supply'] = np.where(df1['day_of_wk'] == 'Wednesday', df1['ZCM ASN QTY Supply'] + df1['ZCM Commits WK1'],
                                       np.where(df1['day_of_wk'] == 'Thursday', df1['ZCM ASN QTY Supply'] + df1['ZCM Commits WK1'],
                                       np.where(df1['day_of_wk'] == 'Friday', df1['ZCM ASN QTY Supply'] + df1['ZCM Commits WK1'],
                                       np.where(df1['day_of_wk'] == 'Saturday', df1['ZCM ASN QTY Supply'] + df1['ZCM Commits WK1'],
                                       np.where(df1['day_of_wk'] == 'Sunday', df1['ZCM ASN QTY Supply'] + df1['ZCM Commits WK1'],
                                       np.where((df1['day_of_wk'] == 'Monday') & (df1['ZCM ASN QTY Supply'] >= df1['ZCM Commits WK1']), df1['ZCM ASN QTY Supply'],
                                       np.where((df1['day_of_wk'] == 'Tuesday') & (df1['ZCM ASN QTY Supply'] >= df1['ZCM Commits WK1']), df1['ZCM ASN QTY Supply'], df1['ZCM Commits WK1'])))))))
df1 = df1.drop(['day_of_wk'], axis = 1)


## 2ND check POINT MARISOL
print("Creating Red Column")
#CREATE RED COLUMN
#THE SKU DC WITH DIFFERENT COMMENTS WILL KEEP ONLY ONE, IF SEVERAL THE WILL BE CONCATANETED
df13 = pd.crosstab(index=df1["SKU DC Key"], columns=df1["CRSD WK"], values=df1["CRSD WK"], aggfunc='count').fillna(0)
df13 = df13.reset_index()
df13.sort_values(by='SKU DC Key', ascending=True, inplace=True)

df13['WK1'] = df13['WK1'].astype(int)
df13['WK2'] = df13['WK2'].astype(int)
df13['WK3'] = df13['WK3'].astype(int)
#df13.to_excel('RED COLUMN TABLE1.xlsx', index=False)
df13['WK1'] = np.where(df13['WK1'] > 0, 'Yes', 'No')
df13['WK2'] = np.where(df13['WK2'] > 0, 'Yes', 'No')
df13['WK3'] = np.where(df13['WK3'] > 0, 'Yes', 'No')
df13[''] = np.where(df13[''] > 0, 'Yes', 'No')

#df13 = df13.drop([''], axis = 1)
df13['Missing CRSD Wk'] = np.where((df13['WK1'] == 'No') & (df13['WK2'] == 'No') & (df13['WK3'] == 'No'), 'All Missing Blank',
                          np.where((df13['WK1'] == 'No') & (df13['WK2'] == 'Yes') & (df13['WK3'] == 'No') & (df13[''] == 'Yes'), 'Missing WK1,Missing WK3',#
                          np.where((df13['WK1'] == 'No') & (df13['WK2'] == 'No') & (df13['WK3'] == 'Yes'), 'All Missing WK3',
                          np.where((df13[''] == 'Yes') & (df13['WK2'] == 'Yes') & (df13['WK3'] == 'No'), 'Missing WK3',         
                          np.where((df13['WK1'] == 'Yes') & (df13['WK3'] == 'Yes') & (df13['WK2'] == 'No'), 'Missing WK2',
                          np.where((df13['WK2'] == 'Yes') & (df13['WK1'] == 'No'), 'Missing WK1', 'Nothing Missing'))))))
#df13.to_excel('RED COLUMN TABLE2.xlsx', index=False)
df1 = pd.merge(df1, df13, on='SKU DC Key', how='left')
df1 = df1.drop(['WK1','WK2','WK3'], axis = 1)

df1['DISTY']=''
df1['SLSOFF2']=''
df1['PSION']=''
df1['COMPLETE']=''
df1['IR_NUMBER']=''

print('Reordering columns')
df1 = df1[["DATE","THEATRE","SHIP_LOC","Trilogy Or Merlin","ORD_TYPE","SLSOFF2","SLSOFF2_DE","BOOKED","CRSD","CRSD Days",
           "Material Avail Date > CRSD","SSD","FSSD","DFF_CRSD","SO","LINE","SO_LINE","SO Line Dropped to DC for Delivery",
           "SKU Short(QTYs On Hand) DBL & CRSD <=+4","SO Short(QTYs On Hand) DBL & CRSD <=+4",
           "SHIP-SET $'s","ORDER $'s","SHIP_SET","PART","Missing CRSD Wk","CRSD WK","TPC SKU/DC/WK Key","SKU DC Key","TPC/ZCM Transmit","MATL_TYPE",
           "ACTUAL_QTY","REMAIN_QTY","Part ASP $","Blended ASP $",
           "EXT Part ASP $","UNIT_VAL","TOT_BL","TOP-FILE $'s","SOLDTO_NO","SOLD_TO","SHIP_TO","SHIP_DESC","HOLD",
           "DISTY","DISTY_1","TOTAL_ORD","FAMILY","PROD_LINE","CORE","COMPLETE","PO","SOURCE.","Sourcing Rule","XFR","SOURCE_2",
           "STATUS","CARRIER","SHIP_MTHD","IR_NUMBER","DAYS_OLD","DAYS_OLD2","BUYER.","PLANNER CODE","BUYER","INVC_QTY",
           "REMAIN_INV","DISTY2","FSSD_CRSD","HOLD_2","REASON","SHIP_QTY",
           "QOH","Oracle Intransit(Trilogy IR/ISO)","Oracle Intransit(Trilogy=>Merlin or Merlin=>Trilogy)","ZCM ASN QTY Supply",
           "ZCM Commits WK1","ZCM Commits WK2","ZCM Commits WK3","ZCM Commits WK4","ZCM Commits WK5","WK1 TPC Receipt QTY",
           "Valid TPC Supply","TPC Commits WK1","TPC Commits WK2","TPC Commits WK3","TPC Commits WK4",
           "ZCM ASN/Commit Net WK1 Supply",
           "ZCM ASN QTY and WK1 Net Supply","ZCM Net Commits WK2","ZCM Net Commits WK3","Valid TPC Net Supply WK1",
           "TPC Net Commits WK2","TPC Net Commits WK3","Missin Commits TPC/ZCM WK2","Net Supply capture by CRSD SKU/DC/WK",
           "Missing ASN Supply Due to CRSD WK","Missing Commit Supply Due to CRSD WK","Merlin/Trilogy Missing Supply Total",
           "ORG_BOOK","SKU/DC Total Supply(OH only)","QTY Short to Sales Orders(CUM) Only INV","QTY Short to Sales Orders(NON-CUM) Only INV",
           "CRSD_WEEK","CRSD_MONTH","CRSD_QTR","SSD_WEEK","SSD_MONTH","SSD_QTR","PSION","COUNTRY","DELINQ","ALIGN","BOOK_WEEK",
           "BU","SYSTEM","CREATED_BY","DFF_CRSD2","BOOK_CRSD","BOOK_CRSD2","DAYS_OLD3","SUB_FAMILY","CODE_NAME","THEATRE1",
           "EVM_AIT","HOLD_3","INCO","EVM_AIT2","DELINQ_SSD","DAYS_OLD4","PAY_TERMS","FISC_ZCRD","FISC_SSD","BUSINESS","PRIORITY",
           "STATE","SUB_LINE","EXT Part ASP $ (QTY Short to Sales Orders(NON-CUM) Only INV)","SKU/DC Total Supply(On Hand+ALL ASN)",
           "QTY Short to Sales Orders(CUM) (On Hand+ALL ASN","QTY Short to Sales Orders(NON-CUM) On Hand+ALL ASN",
           "EXT Part ASP $ (QTY Short to Sales Orders(NON-CUM)) On Hand+ALL ASN","SKU/DC Total Supply ALL ASNs+Commits(No Missing Commits)",
           "QTY Short to Sales Orders(CUM) including Missing Commits+Commits+ALL ASNs","QTY Short to Sales Orders(NON-CUM) w/ Commits+Missing Commits+ALL ASNs",
           "EXT Part ASP $ (QTY Short to Sales Orders(NON-CUM)) INV w/ Commits+Missing Commits+ALL ASNs","SKU/DC Total Supply(OH+ALL ASNs+all 3wk Commtsh)",
           "QTY Short to Sales Orders(CUM) OH+ALL ASNs+all 3wk Commts","QTY Short to Sales Orders(NON-CUM) OH+ALL ASNs+all 3wk Commts",
           "EXT Part ASP $ (QTY Short to Sales Orders(NON-CUM)) OH+ALL ASNs+all 3wk Commts","Xplore Make items"]]

#print('Done')
df1.sort_values(by=['SKU DC Key','CRSD'],inplace=True)

#CY NEW
#Auxiliar to keep only first rows, its false in AUX1
#since AUX1 answers to the question, is this a duplicate row?
df1['skuduplicates']=df1.duplicated(subset=['SKU DC Key']) #FALSE FOR ALL FIRST APPEARENCE of SKU DC Key,TRUE FOR DUPLICATES
#df1['AUX1']=np.where(df1['skuduplicates']== False,df1['skuduplicates'],'')
#DUPLICATES FOR SKUDC+WK
df1['skuduplicates2']=df1.duplicated(subset=['TPC SKU/DC/WK Key'])

df1['AUX'] = np.where((df1['Missing CRSD Wk'] == 'Missing WK1,Missing WK3') & (df1['skuduplicates2']== False) & (df1['CRSD WK']== 'WK2'), 
                            df1['ZCM Commits WK1'],
             np.where((df1['Missing CRSD Wk'] == 'All Missing WK3') & (df1['skuduplicates2']== False) & (df1['CRSD WK']== 'WK3'),
                      df1['ZCM Commits WK1'] + df1['ZCM Commits WK2'],
             np.where((df1['Missing CRSD Wk'] == 'All Missing Blank') & (df1['skuduplicates']== False),
                      df1['ZCM Commits WK1'] + df1['ZCM Commits WK2'] + df1['ZCM Commits WK3'],
             np.where((df1['Missing CRSD Wk'] == 'Missing WK1') & (df1['skuduplicates2']== False) & (df1['CRSD WK']== 'WK2'),
                      df1['ZCM Commits WK1'],
             np.where((df1['Missing CRSD Wk'] == 'Missing WK2') & (df1['skuduplicates2']== False) & (df1['CRSD WK']== 'WK3'),
                      df1['ZCM Commits WK2'],
             np.where((df1['Missing CRSD Wk'] == 'Missing WK3') & (df1['skuduplicates2']== False) & (df1['CRSD WK']== ''),
                      df1['ZCM Commits WK3'],
            np.where((df1['Missing CRSD Wk'] == 'Missing WK1,Missing WK3') & (df1['skuduplicates2']== False) & (df1['CRSD WK']== ''),
                      df1['ZCM Commits WK3'],0)))))))
#COLUMN CY MUTIPLE VLOOKUPS DEPENDING ON RED COLUMN
df1['Missing Commit Supply Due to CRSD WK'] = pd.to_numeric(df1['AUX'])
df1['Missing Commit Supply Due to CRSD WK'].fillna(0, inplace=True)


#ZCM ASN QTY and WK1 Net Supply CP COLUMN
df1['ZCM ASN QTY and WK1 Net Supply'] = np.where( (df1['skuduplicates2']== False) & (df1['CRSD WK'] == 'WK1'),
                                                           df1['ZCM ASN/Commit Net WK1 Supply'],0 )

#ZCM ASN QTY and WK1 Net Supply CQ COLUMN
df1['ZCM Net Commits WK2'] = np.where( (df1['skuduplicates2']== False) & (df1['CRSD WK'] == 'WK2'),
                                                           df1['ZCM Commits WK2'],0 )
#ZCM ASN QTY and WK1 Net Supply CR COLUMN
df1['ZCM Net Commits WK3'] = np.where( (df1['skuduplicates2']== False) & (df1['CRSD WK'] == 'WK3'),
                                                           df1['ZCM Commits WK3'],0 )
#Net Supply capture by CRSD SKU/DC/WK CW COLUMN  
#CP+CQ+CR+CS+CT+CU
df1['Net Supply capture by CRSD SKU/DC/WK'] = np.where( df1['skuduplicates2']== False,
                                                           df1['ZCM ASN QTY and WK1 Net Supply'] + 
                                                           df1['ZCM Net Commits WK2'] + df1['ZCM Net Commits WK3'],0 )
df1['ZCM ASN QTY Supply'] = pd.to_numeric(df1['ZCM ASN QTY Supply'])
df1['Missing Commit Supply Due to CRSD WK'] = pd.to_numeric(df1['Missing Commit Supply Due to CRSD WK'])

###########NEW
# COLUMN CX =+IF(Z4<>"Nothing Missing",CC4,0)
#bring the ZCM ASN QTY Supply to the Missing ASN Supply Due to CRSD WK column
#Nothing Missing ACCORDING TO RED COLUMN VALUES

df1['Missing ASN Supply Due to CRSD WK'] = np.where((df1['Missing CRSD Wk'] == 'All Missing Blank') & (df1['skuduplicates']== False) | (df1['Missing CRSD Wk'] == 'All Missing WK3') & (df1['skuduplicates2']== False) & (df1['CRSD WK']=='WK3') 
                                                    | (df1['Missing CRSD Wk'] == 'Missing WK1,Missing WK3') & (df1['skuduplicates2']== False) |
                                                    (df1['Missing CRSD Wk'] == 'Missing WK1') & (df1['skuduplicates2']== False) & (df1['CRSD WK']=='WK2') | 
                                                    (df1['Missing CRSD Wk'] == 'Missing WK2') & (df1['skuduplicates2']== False)& (df1['CRSD WK']=='WK3') |
                                                    (df1['Missing CRSD Wk'] == 'Missing WK3') & (df1['skuduplicates2']== False) & (df1['CRSD WK']==''), 
                                                    df1['ZCM ASN QTY Supply'], 0)

df1['Missing ASN Supply Due to CRSD WK'] = pd.to_numeric(df1['Missing ASN Supply Due to CRSD WK'])



#COLUMN CZ Last column of green/orange bucket, copied formulas from yesterday's file
#cz =IF(OR(AA4="WK2",AA4="WK3",AA4=""),CY4,IF(CY4-CX4<0,CX4,CY4))
df1['Merlin/Trilogy Missing Supply Total'] = np.where( df1['CRSD WK'] != 'WK1',
                                            df1['Missing Commit Supply Due to CRSD WK'],
                                            np.where( df1['Missing Commit Supply Due to CRSD WK'] - df1['Missing ASN Supply Due to CRSD WK'] < 0,
                                            df1['Missing ASN Supply Due to CRSD WK'], df1['Missing Commit Supply Due to CRSD WK'])) 

df1['Merlin/Trilogy Missing Supply Total'] = np.where( (df1['Merlin/Trilogy Missing Supply Total'] > 0) & 
                                                      ((df1['Missing CRSD Wk'] == 'Missing WK2') | (df1['Missing CRSD Wk'] == 'Missing WK3')),
                                            df1['Missing Commit Supply Due to CRSD WK'],df1['Merlin/Trilogy Missing Supply Total'])

#print(len(df1) & "number of records")
dfSourcingRule.columns = dfagile.columns
agileupdate = dfagile.append(dfSourcingRule)  
agileupdate.to_excel('Agile list1.xlsx', index=False) #fix automatized name
c=(time.time() - start_time)/60
print("--- %s minutes to create EVM BL---" % c)

#print(len(df1))
print('Writing file')
strings = time.strftime("%m.%d.%Y")
strings= 'EVM ' + strings
b=("%s.xlsx" % strings)
df1=df1.drop(['skuduplicates','skuduplicates2','AUX'], axis=1)

df1.to_excel(b, sheet_name= 'COMBINED',index=False,startrow=2)
c=(time.time() - start_time)/60
print("--- %s minutes to create EVM BL and write---" % c)

Reading the Current Backlog Combined Data
Filtering file for EVM
Inserting columns
Reading the In Play Report
Reading ZCM Item Report
Reading Fill Rate
Reading the Sourcing Rules for EVM
Reading Agile List
Reading Inter-Org Intransit
Reading the shipments
ASN file created
Reading All Commit_ZCM
Reading Xplore Items
Creating Red Column
Reordering columns
--- 22.586510423819224 minutes to create EVM BL---
Writing file
--- 37.04956984917323 minutes to create EVM BL and write---


In [2]:
a=fnmatch.filter(os.listdir('.'), 'All Commit_ZCM*')
a=a[0]
#print(a)
#print(b)
pathtc = "C:\\Users\\MJ7255\\Desktop\\Backlog\\Backlog Report SAM\\Backlog Report\\Untitled Folder\\" + a
pathf = "C:\\Users\\MJ7255\\Desktop\\Backlog\\Backlog Report SAM\\Backlog Report\\Untitled Folder\\" + b
wbtc = xw.Book(pathtc)
wbf = xw.Book(pathf)
wstc = wbtc.sheets(1)

wstc.api.Copy(After=wbf.sheets(1).api)
wbf.save()
wbtc.app.quit()